Intended use case: given a file with assembled traces, open it, copy data, open new data, append, and write the extended dataset into a new file. Specific use: to add LFP+LabView-only dataset to assembled_traces CHR2 file

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import numpy as np
import pandas as pd
import labrotation.file_handling as fh
import datadoc_util as ddutil
from labrotation import two_photon_session as tps
import h5py
from datetime import datetime
from datetime import timedelta
import matplotlib.pyplot as plt
import pyabf
import matlab.engine

In [ ]:
def get_datetime_for_fname():
    now = datetime.now()
    return f"{now.year:04d}{now.month:02d}{now.day:02d}-{now.hour:02d}{now.minute:02d}{now.second:02d}"

In [ ]:
def create_dt(t):
    t1 = t[1:]
    t0 = t[:-1]
    dt = np.zeros(len(t))
    dt[1:] = t1 - t0
    dt[0] = dt[1]  # assume same step size to avoid 0
    return dt
def create_totdist_abs(speed, dt):
    totdist_abs = np.zeros(len(speed))
    totdist_abs[0] = speed[0]*dt[0]
    for i in range(1, len(totdist_abs)):
        totdist_abs[i] = totdist_abs[i-1] + abs(speed[i]*dt[i])
    return totdist_abs

In [ ]:
env_dict = dict()
if not os.path.exists("./.env"):
    print(".env does not exist")
else:
    with open("./.env", "r") as f:
        for line in f.readlines():
            l = line.rstrip().split("=")
            env_dict[l[0]] = l[1]
print(env_dict.keys())

In [ ]:
if "SERVER_SYMBOL" in env_dict.keys():
    SERVER_SYMBOL = env_dict["SERVER_SYMBOL"]
else:
    SERVER_SYMBOL = "E"
    print(f"Server symbol not found in .env file. Setting it to {SERVER_SYMBOL}")

In [ ]:
matlab_2p_folder = env_dict["MATLAB_2P_FOLDER"]

In [ ]:
output_dir = fh.open_dir("Choose export directory for assembled traces!")

In [ ]:
dset_folder = fh.open_dir("Choose directory to import results from")

In [ ]:
ddoc = ddutil.DataDocumentation(env_dict["DATA_DOCU_FOLDER"])
ddoc.loadDataDoc()
ddoc.setDataDriveSymbol(SERVER_SYMBOL)
ddoc.checkFileConsistency()

In [ ]:
existing_uuids = []
for root, folders, files in os.walk(dsets_dir):
    for file in files:
        if os.path.splitext(file)[-1] == ".h5":
            original_fname = file.split("_segmented")[0]  # files should be named [original lfp file name]_segmented.h5
            #print(original_fname)
            #print(ddoc.getUUIDForFile(original_fname + ".abf"))
            existing_uuids.append(ddoc.getUUIDForFile(original_fname + ".abf"))

## List recordings to be added 

In [ ]:
df_new_recordings = ddoc.getRecordingsWithExperimentType(["chr2_ctl_bilat", "chr2_sd_bilat", "chr2_szsd_bilat"])
# do the filtering by mouse
dset_dict = {"no_nikon": ["WEZ-8946", "WEZ-8960"]}

df_new_recordings = df_new_recordings[df_new_recordings["mouse_id"].isin(dset_dict["no_nikon"])]

In [ ]:
for i, row in df_new_recordings.iterrows():
    if row.uuid in existing_uuids:
        print(f"{row.uuid}: {row.mouse_id}\t{row.lfp}")

## Manually save them

In [ ]:
uuid = "202c1dde3e5d4c07b9033d8c6f82b7ef"

In [ ]:
files = ddoc.getSessionFilesForUuid(uuid).iloc[0]

In [ ]:
fold = files.folder
lfp_fpath = os.path.join(fold, files["lfp"])
lv_fpath = os.path.join(fold, files["labview"])
lv_times_fpath = os.path.join(fold, os.path.splitext(files["labview"])[0]+"time.txt")

In [ ]:

lv_t = []  # col 0 of ...time.txt files

with open(lv_times_fpath, "r") as f:
    for line in f.readlines():
        lv_t.append(float(line.split("\t")[0]))
lv_t = np.array(lv_t)
#lv_t = lv_t - lv_t[0]  # set to 0 starting time
#lv_t = lv_t/1000.  # convert to s

lv_rounds = []  # col 0
lv_speed = []  # col 1
lv_totdist = []  # col 2
lv_distancepr = []  # col 3
lv_stripes = []  # col 6
lv_stripespr = []  # col 7
with open(lv_fpath, "r") as f:
    for line in f.readlines():
        if len(line.strip().split("\t")) == 20:
            lv_rounds.append(int(line.strip().split("\t")[0]))
            lv_speed.append(float(line.strip().split("\t")[1]))
            lv_totdist.append(float(line.strip().split("\t")[2]))
            lv_distancepr.append(float(line.strip().split("\t")[3]))
            lv_stripes.append(float(line.strip().split("\t")[6]))
            lv_stripespr.append(float(line.strip().split("\t")[7]))
            
        else:
            print(len(line.strip().split("\t")))
lv_speed = np.array(lv_speed)
lv_rounds = np.array(lv_rounds)
lv_totdist = np.array(lv_totdist)
lv_distancepr = np.array(lv_distancepr)
lv_stripes = np.array(lv_stripes)
lv_stripespr = np.array(lv_stripespr)

lv_t = lv_t[:len(lv_speed)]  # cut out last, incomplete entry

# cut if lv_t shorter
lv_speed = lv_speed[:len(lv_t)]
lv_rounds = lv_rounds[:len(lv_t)]
lv_totdist = lv_totdist[:len(lv_t)]
lv_distancepr = lv_distancepr[:len(lv_t)]
lv_stripes = lv_stripes[:len(lv_t)]
lv_stripespr = lv_stripespr[:len(lv_t)]

assert len(lv_t) == len(lv_speed)

In [ ]:
# run part of matlab pipeline
belt_struct = {"speed":lv_speed, "distance": lv_totdist, "round":lv_rounds, "distancePR": lv_distancepr, "stripes": lv_stripes, "stripesPR": lv_stripespr, "time": lv_t}
eng = matlab.engine.start_matlab()
m2p_path = eng.genpath(matlab_2p_folder)
eng.addpath(m2p_path, nargout=0)
belt_struct_proc = eng.beltCorrectWithoutNikon(belt_struct, nargout=1)

In [ ]:
# convert to numpy 
belt_dict = dict()
for key in belt_struct_proc:   
    belt_dict[key] = np.squeeze(np.array(belt_struct_proc[key]))
belt_dict["time_s"] = belt_dict["time"]/1000.
belt_dict["time_s"] = belt_dict["time_s"] - belt_dict["time_s"][0]

In [ ]:
lfp_file = pyabf.ABF(lfp_fpath)

In [ ]:
lfp_file.setSweep(sweepNumber=0, channel=0)
lfp_t = lfp_file.sweepX
lfp_y = lfp_file.sweepY

lfp_file.setSweep(sweepNumber=0, channel=1)
lfp_loco = lfp_file.sweepY
lfp_t = lfp_t*1.0038
lfp_t = lfp_t - lfp_t[0]  # shift to 0

In [ ]:
(belt_dict["time_s"][-1]-lfp_t[-1])/2.

In [ ]:
bl_length = 300  # 5 min
stim_length = 4  # 4 sec

### Replicate some steps from the matlab processing step

In [ ]:
exp_type = ddoc.getExperimentTypeForUuid(uuid)
mouse_id = ddoc.getMouseIdForUuid(uuid)
win_type = "None"

In [ ]:
offset = 0.8

### Match two locomotion channels

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.plot(belt_dict["time_s"]-offset, belt_dict["speed"])
plt.plot(lfp_t, lfp_loco-2.5)
plt.xlim((0, 100))
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.plot(belt_dict["time_s"]-offset, belt_dict["speed"])
plt.plot(lfp_t, lfp_loco-2.5)
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.plot(belt_dict["time_s"]-offset, belt_dict["speed"])
plt.plot(lfp_t, lfp_y)
plt.vlines(x=[bl_length, bl_length+stim_length], ymin=-1, ymax=1, color="red")
#plt.xlim((295, 390))
#plt.xlim((0, 20))
plt.show()
# TODO: come up with a constant offset, apply it to all recordings and check roughly if they match. Then decide whether use LFP or LabView (make sure labview > 10 min, so we can use loco quantities)
# TODO: then assemble the dataset, include in loco analysis

In [ ]:
fig = plt.figure(figsize=(18,18))
plt.plot(belt_dict["time_s"]-offset, belt_dict["speed"])
plt.plot(lfp_t, lfp_y)
plt.vlines(x=[bl_length, bl_length+stim_length], ymin=-1, ymax=1, color="red")
plt.xlim((295, 390))
#plt.xlim((0, 20))
plt.show()

In [ ]:
output_fpath = os.path.join(dsets_dir, os.path.splitext(os.path.split(lfp_fpath)[-1])[0] + "_segmented.h5")
print(output_fpath)

In [ ]:
dt = create_dt(belt_dict["time_s"])
totdist_abs = create_totdist_abs(belt_dict["speed"], dt)

In [ ]:
with h5py.File(output_fpath, "w") as hf:
    hf.attrs["uuid"] = uuid
    hf.attrs["stim_start"] = bl_length
    hf.attrs["poststim_start"] = bl_length + stim_length
    hf.attrs["exp_type"] = exp_type
    hf.attrs["win_type"] = win_type
    hf.attrs["mouse_id"] = mouse_id
    
    hf.create_dataset("lfp_mov_t", data=lfp_t)
    hf.create_dataset("lfp_mov_y", data=lfp_loco)    
    hf.create_dataset("lfp_t", data=lfp_t)    
    hf.create_dataset("lfp_y", data=lfp_y) 
    
    hf.create_dataset("lv_dist", data=belt_dict["distancePR"])     
    hf.create_dataset("lv_dt", data=dt) 
    hf.create_dataset("lv_speed", data=belt_dict["speed"]) 
    hf.create_dataset("lv_running", data=belt_dict["running"]) 
    hf.create_dataset("lv_t_s", data=belt_dict["time_s"]-offset) # match to lfp
    hf.create_dataset("lv_totdist", data=belt_dict["distance"])  # totdist - where does it come from? Is it distance?
    hf.create_dataset("lv_totdist_abs", data=totdist_abs) 
    hf.create_dataset("lv_rounds", data=belt_dict["round"])
    hf.create_dataset("lv_stripesPR", data=belt_dict["round"])
    hf.create_dataset("lv_distancePR", data=belt_dict["distancePR"])
    
    
    
    

### Run until this point, then go back to specify new UUID to add

# Open all, calculate 15Hz dataset (compatible with recordings including Nikon), save as single file

In [ ]:
dict_attrs = dict()
dict_data = dict()

for root, dirs, files in os.walk(dset_folder):
    for file in files:
        fpath = os.path.join(root, file)
        assert os.path.exists(fpath)
        with h5py.File(fpath, "r") as hf:
            dict_current_attrs = dict()
            for key in hf.attrs.keys():
                dict_current_attrs[key] = hf.attrs[key]
            uuid = dict_current_attrs["uuid"]
            dict_attrs[uuid] = dict_current_attrs
            
            dict_current_data = dict()
            for key in hf.keys():
                dict_current_data[key] = hf[key][:]
            dict_data[uuid] = dict_current_data

In [ ]:
dict_dset = dict()
for uuid in dict_data.keys():
    print(uuid)
    t_last = dict_data[uuid]["lv_t_s"][-1]
    t_15hz = np.arange(0, t_last, 1/15.)

    # lfp starts at 0, change it so that labview starts at 0
    lv_t_s = dict_data[uuid]["lv_t_s"]
    lfp_t = dict_data[uuid]["lfp_t"]
    lfp_mov_t = dict_data[uuid]["lfp_mov_t"]  # should be same as lfp_t
    
    offset = lv_t_s[0]
    lfp_t = lfp_t - offset
    lfp_mov_t = lfp_mov_t - offset
    lv_t_s = lv_t_s - offset
    
    downsample_indices = np.searchsorted(lv_t_s, t_15hz, side="left")
    # shift first time point to 0 (should not do anything as first entry should be 0)
    offset = lv_t_s[downsample_indices[0]]
    lv_t_s = lv_t_s - offset
    lfp_t = lfp_t - offset
    lfp_mov_t = lfp_mov_t - offset
    
    # define downsampled data, use legacy "scn" notation ("scanner time frame")
    tsscn = lv_t_s[downsample_indices]
    lv_dt_scn = create_dt(tsscn)
    
    lv_dist_scn = dict_data[uuid]["lv_dist"][downsample_indices]
    lv_rounds_scn = dict_data[uuid]["lv_rounds"][downsample_indices]
    lv_running_scn = dict_data[uuid]["lv_running"][downsample_indices]
    lv_speed_scn = dict_data[uuid]["lv_speed"][downsample_indices]
    lv_totdist_scn = dict_data[uuid]["lv_totdist"][downsample_indices]
    lv_totdist_abs_scn = dict_data[uuid]["lv_totdist_abs"][downsample_indices]
    
    dict_uuid = dict()
    dict_uuid["lfp_mov_t"] = lfp_mov_t
    dict_uuid["lfp_mov_y"] =  dict_data[uuid]["lfp_mov_y"]
    dict_uuid["lfp_t"] = lfp_t
    dict_uuid["lfp_y"] = dict_data[uuid]["lfp_y"]
    dict_uuid["lv_dist"] = lv_dist_scn
    dict_uuid["lv_dt"] = lv_dt_scn
    dict_uuid["lv_rounds"] = lv_rounds_scn
    dict_uuid["lv_running"] = lv_running_scn
    dict_uuid["lv_speed"] = lv_speed_scn
    dict_uuid["lv_t_s"] = tsscn
    dict_uuid["lv_totdist"] = lv_totdist_scn
    dict_uuid["lv_totdist_abs"] = lv_totdist_abs_scn
    
    dict_dset[uuid] = dict_uuid
    

In [ ]:
dict_attrs["4b1ee9d07534439a8bc037e6771253f7"]

In [ ]:
#check dataset lengths
for uuid in dict_dset.keys():
    lfp_len = len(dict_dset[uuid]["lfp_mov_t"])
    lv_len = len(dict_dset[uuid]["lv_t_s"])
    for dname in dict_dset[uuid].keys():
        if "lv_" in dname:
            assert len(dict_dset[uuid][dname]) == lv_len
        else:
            assert len(dict_dset[uuid][dname]) == lfp_len

In [ ]:
# save to single file of assembled traces
out_fpath = os.path.join(output_dir, f"assembled_traces_bilat_{get_datetime_for_fname()}.h5")
with h5py.File(out_fpath, "w") as hf:
    for uuid in dict_dset.keys():
        uuid_grp = hf.create_group(uuid)

        
        lv_t_s = dict_dset[uuid]["lv_t_s"]
        lfp_t = dict_dset[uuid]["lfp_t"]
        
        t_stim = dict_attrs[uuid]["stim_start"]
        t_poststim = dict_attrs[uuid]["poststim_start"]
        
        i_stim_lv = np.searchsorted(lv_t_s, t_stim)
        i_poststim_lv =  np.searchsorted(lv_t_s, t_poststim)
        i_stim_lfp = np.searchsorted(lfp_t, t_stim)
        i_poststim_lfp = np.searchsorted(lfp_t, t_poststim)
        
        # add attributes
        uuid_grp.attrs["uuid"] = uuid   
        uuid_grp.attrs["mouse_id"] = dict_attrs[uuid]["mouse_id"]
        uuid_grp.attrs["window_type"] = dict_attrs[uuid]["win_type"]
        uuid_grp.attrs["exp_type"] = dict_attrs[uuid]["exp_type"]
        uuid_grp.attrs["break_points"] = np.array([0, i_stim_lv, i_poststim_lv])
        print(np.array([0, i_stim_lv, i_poststim_lv]))
        print(uuid_grp.attrs["break_points"])
        uuid_grp.attrs["break_points_lfp"] = np.array([np.searchsorted(lfp_t, 0.),  i_stim_lfp, i_poststim_lfp])
        uuid_grp.attrs["has_lfp"] = True
        uuid_grp.attrs["i_stim_begin_frame"] = i_stim_lv
        uuid_grp.attrs["i_stim_begin_frame"] = i_poststim_lv - 1
        uuid_grp.attrs["n_bl_frames"] = i_stim_lv
        uuid_grp.attrs["n_am_frames"] = len(lv_t_s) - i_poststim_lv
        uuid_grp.attrs["n_frames"] = len(lv_t_s)
        uuid_grp.attrs["n_lfp_mov_steps"] = len(lfp_mov_t)
        uuid_grp.attrs["n_lfp_steps"] = len(lfp_t)
        uuid_grp.attrs["recording_break_points"] = np.array([0])
        uuid_grp.attrs["recording_break_points_lfp"] = np.array([0])
        uuid_grp.attrs["session_uuids"] = [uuid]
        #  add data
        for dset_name in dict_dset[uuid].keys():
            uuid_grp.create_dataset(dset_name, data=dict_dset[uuid][dset_name])

# Open all recordings, calculate metrics

In [ ]:
dict_attrs = dict()
dict_data = dict()

for root, dirs, files in os.walk(dset_folder):
    for file in files:
        fpath = os.path.join(root, file)
        assert os.path.exists(fpath)
        with h5py.File(fpath, "r") as hf:
            dict_current_attrs = dict()
            for key in hf.attrs.keys():
                dict_current_attrs[key] = hf.attrs[key]
            uuid = dict_current_attrs["uuid"]
            dict_attrs[uuid] = dict_current_attrs
            
            dict_current_data = dict()
            for key in hf.keys():
                dict_current_data[key] = hf[key][:]
            dict_data[uuid] = dict_current_data

In [ ]:
dict_attrs

### Calculate pre-stim, post stim metrics

In [ ]:
event_uuids = []
mouse_ids = []
window_types = []
exp_types = []
segment_types = []
segment_lengths = []
totdists = []
totdists_abs = []
runnings = []
totdists_norm = []
totdists_abs_norm = []

#df_stats["totdist_abs_norm"] = 10000*df_stats["totdist_abs"]/df_stats["segment_length"]  # for totdist_abs, can use 4500 as length

for uuid in dict_attrs.keys():
    mouse_id = dict_attrs[uuid]["mouse_id"]
    win_type = dict_attrs[uuid]["win_type"]
    exp_type = dict_attrs[uuid]["exp_type"]
    
    # separate loco data into pre-stim and post-stim
    dict_pre = dict()
    dict_post = dict()
    t_stim = dict_attrs[uuid]["stim_start"]
    t_poststim = dict_attrs[uuid]["poststim_start"] 
    t_data = dict_data[uuid]["lv_t_s"]
    
    i_pre = t_data < t_stim
    pre_length = np.sum(i_pre)
    
    # create matching post segment
    i_first_post = np.argmax(t_data >= t_poststim)
    i_post = np.zeros(t_data.shape)
    i_post[i_first_post:i_first_post+pre_length] = 1
    i_post = np.bool_(i_post)
    post_length = np.sum(i_post)
    if not pre_length == post_length:
        print(pre_length)
        print(post_length)
    #i_post = np.logical_and(t_data >= t_poststim, t_data < )
    
    post_length = pre_length
    for key in dict_data[uuid].keys():
        if "lv_" in key:
            data_arr = dict_data[uuid][key]
            dict_pre[key] = data_arr[i_pre]
            dict_post[key] = data_arr[i_post]
    # calculate values
    totdist_pre = dict_pre["lv_totdist"][-1] - dict_pre["lv_totdist"][0]
    totdist_post = dict_post["lv_totdist"][-1] - dict_post["lv_totdist"][0]
    
    totdist_abs_pre = dict_pre["lv_totdist_abs"][-1] - dict_pre["lv_totdist_abs"][0]
    totdist_abs_post = dict_post["lv_totdist_abs"][-1] - dict_post["lv_totdist_abs"][0]
    
    running_pre = np.sum(dict_pre["lv_running"])*4500/pre_length  # normalize to other data
    running_post = np.sum(dict_pre["lv_running"])*4500/post_length
    
    totdist_norm_pre = 10000*totdist_pre/4500.
    totdist_norm_post = 10000*totdist_post/4500.
    
    totdist_abs_norm_pre = 10000*totdist_abs_pre/4500.
    totdist_abs_norm_post = 10000*totdist_abs_post/4500.
    
    # add pre
    event_uuids.append(uuid)
    mouse_ids.append(mouse_id)
    window_types.append(win_type)
    exp_types.append(exp_type)
    segment_types.append("baseline")  # "post-stimulation"
    segment_lengths.append(4500)
    totdists.append(totdist_pre)
    totdists_abs.append(totdist_abs_pre)
    runnings.append(running_pre)
    totdists_norm.append(totdist_norm_pre)
    totdists_abs_norm.append(totdist_abs_norm_pre)    
    # add post
    event_uuids.append(uuid)
    mouse_ids.append(mouse_id)
    window_types.append(win_type)
    exp_types.append(exp_type)
    segment_types.append("post-stimulation")
    segment_lengths.append(4500)
    totdists.append(totdist_post)
    totdists_abs.append(totdist_abs_post)
    runnings.append(running_post)
    totdists_norm.append(totdist_norm_post)
    totdists_abs_norm.append(totdist_abs_norm_post)  

In [ ]:
df_metrics = pd.DataFrame(data={"event_uuid": event_uuids, "mouse_id": mouse_ids, "window_type": window_types, "exp_type": exp_types, "segment_type": segment_types,
                  "segment_length": segment_lengths, "totdist": totdists, "totdist_abs": totdists_abs, "running": runnings, "totdist_norm": totdists_norm, "totdist_abs_norm": totdists_abs_norm})

In [ ]:
df_metrics.to_excel(f"D:\PhD\Data\\bilat_loco_metrics_{get_datetime_for_fname()}.xlsx", index=False)

In [ ]:
df_metrics